#hide_input
# "Using Deep Learning to Classify Airliner Flight Profiles for Post-Flight Analysis"
> "Capstone Project for Master of Science in Data Science University of Wisconsin"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: true
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

# Using Deep Learning to Classify Airliner Flight Profiles for Post-Flight Analysis

## Capstone Project for Master of Science in Data Science
### University of Wisconsin – Oshkosh
### Oshkosh, WI
### 2019

This project suggests an automatic way for post-flight analysts to undertake classification of flight profiles into useful versus non-useful classes. Instead of using the traditional algorithms for time-series classification, this work makes use of a relatively new approach: Before classifying, first transform a time-series into an image. This allows for the application of a well-developed set of algorithms from the area of computer vision. In this project, we perform a comparison of a number of these transformation techniques in terms of their associated image classification performance. We apply each transformation technique to the time-series dataset in turn, train a Convolutional Neural Network to do classification, and record the performance. Then we select the most performant transformation technique (a simple line plot that got a 100% F1-score) and use it in the rest of the analysis pipeline.

The pipeline consists of three models. The first model classifies flight profiles into developed (useful) and non-developed (non-useful) profiles. The second model performs multi-label classification on the developed profiles from the first model. The labels reflect whether a profile has canonical climb/cruise/descent segments. The last model classifies flight profiles with canonical cruise segments into classes that have extended cruises (useful) and shorter cruises (non-useful).

Next, we prepare a significant unlabeled test dataset, consisting of data points that have never been seen by any of the models. We construct an end-to-end analytic inference process to simulate a production system, apply it to the test dataset, and obtain impressive results. Finally, we make recommendations to post-flight and other interested analysts.

Keywords: Deep learning, Time series, Image Classification, CNN, RNN, Flight path

## 1 BACKGROUND

One of the important operational characteristics of a commercial airliner is its flight path. This term needs qualification though. The *lateral* flight path is the track projected onto a flat earth from above. To visualize the lateral flight path one can plot longitude versus latitude as flight time progresses. The *vertical* flight path, on the other hand, is the altitude profile (viewed from the side). This may be visualized by plotting altitude versus flight time. This project will focus on the vertical flight path. When we use the term flight path or flight profile in the rest of the document, we will always refer to the *vertical* flight path.

During normal operation, an airliner has a predictable flight path. After *takeoff*, it climbs as quickly as possible (during the phase known as *climb*) until it reaches a point called the *top of climb*. The pilot then levels off and usually maintains this altitude for most of the flight (straight-and-level flight). This phase of the flight is known as *cruise*. When nearing its destination, a point is reached that is known as *top of descent*. At this point the pilot enters the *descent* phase. Finally, the flight ends during the *landing* of the aircraft.

Post-flight data analysts are often interested in separating useful from non-useful (or less useful) flight paths prior to a specific analysis. In this document, and its associated analysis code, useful profiles will be labeled as *typical* (abbreviated as “typ”) while less useful, non-useful, or anomalous flight paths (for a specific analysis) will be labeled as *non-typical* (abbreviated as “non”). A *typical* flight profile, in the context of this document, has a relatively extended cruise section without changes in altitude (see Figure 1.1). This characteristic will make it useful for certain types of analyses, for example, to estimate hard-to-measure variables like drag and exact angle-of-attack, as well as estimations of the positions of vertical flight control surfaces (even though they are measured). Flight paths could also be considered *non-typical* due to insignificant (i.e. too short) cruise segments and missing data (see Figure 1.2). Note that our definition of usefulness is by no means universal in post-flight analysis.

![Figure 1.1 Typical and Non-typical vertical flight path examples (the non-typical path on the right has steps during cruise)](../images/fig1-1.png "Figure 1.1 Typical and Non-typical vertical flight path examples (the non-typical path on the right has steps during cruise)")

A large airline can operate thousands of flights a day and it is not feasible for the analyst to do this separation/classification in a manual way. What comes to mind next is to construct an algorithm to take on the task. However, it is not straightforward to come up with a traditional algorithm that would discriminate between typical and non-typical flight paths. A promising approach, of course, is to use supervised machine learning and show the model a large enough number of training examples.

![Figure 1.2 Non-typical vertical flight path examples due to insignificant cruise section (left) and missing data (right)](../images/fig1-2.png "Figure 1.2 Non-typical vertical flight path examples due to insignificant cruise section (left) and missing data (right)")

The *predictor points* for this problem are not structured vectors as is common in the case of structured data analysis. Here we have to use a time-series or sequence of scalar-valued predictor points and have the model learn the associated *target point* which is a single categorical “scalar” in each case. The values of the target points will be either *typical* (“typ”) or *non-typical* (“non”). We therefore have a classification problem: Given a flight path (as a scalar-valued time-series), the classifier should choose between typical and non-typical (scalar-valued and categorical).

In the deep learning subfield, it is common to use a *Recurrent Neural Network* (RNN) for this kind of problem. See, for example, Hüsken and Stagge (2003), and also Sun, Di, and Fang (2019). However, the training of an RNN can be challenging due to high demands on computing resources including processing power, processing time, and memory. There is also the vanishing/exploding gradients problem, addressed in various ways, but is often still lurking in the background.

Consider how easy it is for the human visual system to handle this problem, and in a fraction of a second. In fact, this is exactly how analysts often do their classifications manually. This suggests that we might benefit from tapping into the biological mechanisms for analyzing visual data (i.e. images). Recently, some researchers started adopting this insight. See, for example, Wang and Oates (2015a). The essence of this line of thought is the following: Instead of analyzing a sequence of 1-D or scalar-valued *temporal* data points, we transform them into a single 2-D or matrix-valued *spatial* data point. The spatial data point is simply an image which means the time-series signal has been transformed into an image (see Figure 1.3 for an example of a transformation technique). This allows for the application of a large body of relatively well-developed computer vision techniques to the above-stated problem. Most of these techniques center around the *Convolutional Neural Network* (CNN). In summary, the *time-series classification* problem has been converted to an *image classification* problem.

![Figure 1.3 Example of Gramian Angular Summation Field (GASF) transformation of a time-series](../images/fig1-3.png "Figure 1.3 Example of Gramian Angular Summation Field (GASF) transformation of a time-series")

This comparative case study will not attempt to compare the difference between using RNNs and CNNs to solve the flight path classification problem. Instead, it will compare the impact of a number of transformation techniques (to transform the time-series into an image) on the image classification performance. After application of each transformation technique to the training dataset of flight path time-series, a CNN will be trained which will serve as a classifier. The performance of the classifiers will be compared. *Transfer learning* will be used to speed up the training of the CNNs.

## 1.1	VALUE PROPOSITION

This project seeks to provide value in a number of ways:
* Demonstrates how flight profile time-series can be turned into images for more effective classification
* Identifies the best transformation technique for flight path time-series
* Reduces the need for (or does away with) hand classification of flight profiles. This will save significant amounts of time for post-flight analysts.
* Provides an analytic process that can be adopted as a tool by analysts. They can then implement the analytical process in their own preferred technology environment.
* Demonstrates how transfer learning greatly speedup the time to train a CNN neural network for the classification of profiles. This should encourage analysts that might still be skeptical about the use of deep learning for everyday tasks, and save them even more time.
* Demonstrates how post-flight analysis can be undertaken by ordinary analysts. This data is usually considered sensitive by airlines and are not published. A publicly available de-identified source of flight data is used and the project demonstrates how this provides a valuable opportunity for analysts.
* Encourages data scientists to undertake post-flight analyses. This is especially needed in the area of airline safety. In addition, and when allowed by airline policies and pilot unions, post-flight analysis can be a valuable tool in the performance evaluation of pilots. This can have a positive impact on the profitability of an airline.
* Satisfies my personal interest in the analysis of flight data as well as the application of cutting edge analysis techniques in the form of deep learning.

## 1.2 OBJECTIVES

To setup an analytics pipeline for analysts, the foremost objective is to find the best transformation technique to convert flight path time-series into images. The rest of the objectives provide the components for the construction of the pipeline.

### 1.2.1 Transformation Techniques

We will perform a comparison of a number of transformation techniques in terms of their associated image classification performance. To do this we will apply each transformation technique to the cleaned time-series dataset in turn, train a CNN to do classification (using supervised learning), and record the performance. Then we will select the most performant transformation technique and use this technique in the rest of the analysis pipeline. The following transformation techniques will be considered:
* Altitude line plots transformed into an image
* Altitude area plots transformed into an image
* Gramian Angular Summation Field (GASF)
* Gramian Angular Difference Field (GADF)
* Markov Transition Field (MTF)
* Recurrence Plot (RP)

### 1.2.2 Developed/Non-developed Model

The first model in the analytics pipeline will classify flight profiles into developed (useful) and non-developed (non-useful) profiles. We will also consider the use of *anomaly detection* by means of an *auto-encoder* (instead of a classification algorithm) due to a significant class imbalance.

### 1.2.3 Canonical Segments Model